In [21]:

all_symbols = list("/*-+.!@#$%&*()_=+")
symbols = ['.','@']
fixed_pattern = "3010"
max_pass_length = 20
min_pass_length = 7
# -------------- password generator ------------------
with open("pass.lst", 'wb') as f:
    for i in range(min_pass_length,max_pass_length+1):
        for cap in capitals:
            for small in smalls:
                for sym in symbols:
            f.write(cap+)
    

IndentationError: expected an indented block (<ipython-input-21-63b67de1cc36>, line 16)

In [451]:
from string import ascii_lowercase, ascii_uppercase, digits
from itertools import product
from pprint import pprint

symbols = ".@"

key_map = {
    's_letters':ascii_lowercase,
    'u_letters':ascii_uppercase,
    'digits': digits,
    'symbols': symbols
}
all_pass = []

In [494]:
# Using list ---> 10000 loops, best of 3: 147 µs per loop
# Using set ---> 10000 loops, best of 3: 181 µs per loop

# After adding string conversion ---> 1000 loops, best of 3: 283 µs per loop

count = 0
def get_password():
    first_key = 'u_letters'
    needed_keys = ['digits', 'symbols', 's_letters']
    for n in range(6,8):
        for tup in product(needed_keys,repeat=n): # to repreat product nth time
            for p in product(key_map[first_key], *(key_map[key] for key in tup)):
                yield "".join(p)
        

In [495]:
password = get_password()

In [496]:
p = product(*tuple(key_map[key] for key in ['digits','digits']))

In [515]:
password.next()

'A000018'